In [6]:
from dotenv import load_dotenv
load_dotenv()  # This reads the .env file in your project directory
import os
API_KEY = os.environ.get("OPENAI_KEY")

ModuleNotFoundError: No module named 'dotenv'

In [ ]:
import openai

# Set your API key
openai.api_key = API_KEY



prompt = """
Write a bash script that takes a matrix represented as a string with
format '[1,2],[3,4],[5,6]' and prints the transpose in the same format.
"""

response = openai.chat.completions.create(
    model="o1-mini",
    messages=[
        {
            "role": "user",
            "content": prompt
        }
    ]
)

print(response.choices[0].message.content)

APIConnectionError: Connection error.

In [ ]:
response = openai.images.generate(
    prompt="A futuristic city skyline at sunset",
    n=1,
    size="1024x1024"
)

image_url = response.data[0].url
print("Generated Image:", image_url)


In [ ]:
response = openai.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "user", "content": "Write a Python function to check if a number is prime."}
    ]
)

print(response.choices[0].message.content)


# Web Scraping

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv

# Base URL of the website
base_url = 'http://quotes.toscrape.com'

# Starting page
current_page = '/'

# List to store scraped data
quotes_data = []

def scrape_page(page_url):
    # Send a GET request to the page
    response = requests.get(base_url + page_url)

    # Check if the request was successful
    if response.status_code != 200:
        print(f"Failed to retrieve page {page_url}")
        return None

    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all quote containers
    quotes = soup.find_all('div', class_='quote')

    for quote in quotes:
        # Extract the text of the quote
        text = quote.find('span', class_='text').get_text(strip=True)

        # Extract the author's name
        author = quote.find('small', class_='author').get_text(strip=True)

        # Extract tags associated with the quote
        tags = [tag.get_text(strip=True) for tag in quote.find_all('a', class_='tag')]

        # Append the data to the list
        quotes_data.append({
            'Quote': text,
            'Author': author,
            'Tags': ', '.join(tags)
        })

    # Find the 'Next' button to navigate to the next page
    next_button = soup.find('li', class_='next')
    if next_button:
        next_page = next_button.find('a')['href']
        return next_page
    else:
        return None

# Scrape all pages
while current_page:
    print(f"Scraping page: {base_url + current_page}")
    next_page = scrape_page(current_page)
    current_page = next_page

# Define the CSV file headers
headers = ['Quote', 'Author', 'Tags']

# Write data to CSV
with open('quotes.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=headers)
    writer.writeheader()
    for quote in quotes_data:
        writer.writerow(quote)

print(f"Scraping completed. {len(quotes_data)} quotes saved to quotes.csv")



In [ ]:
import pandas as pd

# Read the CSV file
df = pd.read_csv('quotes.csv')

# Display the first few rows
df.head()


In [ ]:
from google.colab import files
files.download('quotes.csv')


In [ ]:
import requests
import pandas as pd

# Define the flight search parameters
origin = "DUB"  # Dublin Airport
destination = "LON"  # London (You can use LON for all London airports)
date_out = "2025-04-11"  # YYYY-MM-DD format
adults = 1  # Number of adult passengers

# Ryanair API endpoint
url = f"https://www.ryanair.com/api/farfnd/v4/oneWayFares?departureAirportIataCode={origin}&arrivalAirportIataCode={destination}&outboundDepartureDateFrom={date_out}&outboundDepartureDateTo={date_out}&adults={adults}"

# Send request
response = requests.get(url)

# Check if request was successful
if response.status_code == 200:
    data = response.json()

    # Extract flights
    flights = []
    for flight in data.get("fares", []):
        flights.append({
            "Departure": flight["outbound"]["departureAirport"]["name"],
            "Arrival": flight["outbound"]["arrivalAirport"]["name"],
            "Departure Time": flight["outbound"]["departureDate"],
            "Price": flight["summary"]["price"]["value"],
            "Currency": flight["summary"]["price"]["currencySymbol"]
        })

    # Convert to DataFrame
    df = pd.DataFrame(flights)
    print(df)

    # Save to CSV
    df.to_csv("ryanair_flights.csv", index=False)
    print("Flights saved to ryanair_flights.csv")

else:
    print(f"Failed to retrieve data. Status Code: {response.status_code}")


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

# Setup Chrome WebDriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service

options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Run browser in the background
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# Open Aer Lingus Flight Search
driver.get("https://www.aerlingus.com/html/en-US/home.html")

# Wait for page to load
time.sleep(3)

# Enter departure and arrival airports
driver.find_element(By.ID, "origin").send_keys("DUB")  # Dublin Airport
driver.find_element(By.ID, "destination").send_keys("LHR")  # London Heathrow

# Select departure date
driver.find_element(By.ID, "departureDate").send_keys("10/02/2025")  # Format: DD/MM/YYYY

# Click search button
driver.find_element(By.ID, "search-button").click()

# Wait for results to load
time.sleep(5)

# Extract flight details
flights = []
flight_elements = driver.find_elements(By.CLASS_NAME, "flight-card")

for flight in flight_elements:
    try:
        departure_time = flight.find_element(By.CLASS_NAME, "departure-time").text
        arrival_time = flight.find_element(By.CLASS_NAME, "arrival-time").text
        price = flight.find_element(By.CLASS_NAME, "price").text

        flights.append({
            "Departure Time": departure_time,
            "Arrival Time": arrival_time,
            "Price": price
        })
    except:
        pass  # Skip if some elements are missing

# Close browser
driver.quit()

# Convert to DataFrame and save
df = pd.DataFrame(flights)
df.to_csv("aerlingus_flights.csv", index=False)
print("Flights saved to aerlingus_flights.csv")
